In [1]:
import pyspark.sql.types as typ #使用时采用typ.DoubleType()的形式
from pyspark.sql.types import DoubleType #直接使用DoubleType()
labels = [
('INFANT_ALIVE_AT_REPORT', typ.DoubleType()),
('BIRTH_PLACE', typ.StringType()),
('MOTHER_AGE_YEARS', DoubleType()),
('FATHER_COMBINED_AGE', typ.DoubleType()),
('CIG_BEFORE', typ.DoubleType()),
('CIG_1_TRI', typ.DoubleType()),
('CIG_2_TRI', typ.DoubleType()),
('CIG_3_TRI', typ.DoubleType()),
('MOTHER_HEIGHT_IN', typ.DoubleType()),
('MOTHER_PRE_WEIGHT', typ.DoubleType()),
('MOTHER_DELIVERY_WEIGHT', typ.DoubleType()),
('MOTHER_WEIGHT_GAIN', typ.DoubleType()),
('DIABETES_PRE', typ.DoubleType()),
('DIABETES_GEST', typ.DoubleType()),
('HYP_TENS_PRE', typ.DoubleType()),
('HYP_TENS_GEST', typ.DoubleType()),
('PREV_BIRTH_PRETERM', typ.DoubleType())
]
# Specifying the schema of the DataFrame
schema = typ.StructType([
    typ.StructField(e[0], e[1], False) for e in labels
])
births = spark.read.csv('inputData/births_train.csv',header=True,schema=schema)
births.show(2)

+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
|INFANT_ALIVE_AT_REPORT|BIRTH_PLACE|MOTHER_AGE_YEARS|FATHER_COMBINED_AGE|CIG_BEFORE|CIG_1_TRI|CIG_2_TRI|CIG_3_TRI|MOTHER_HEIGHT_IN|MOTHER_PRE_WEIGHT|MOTHER_DELIVERY_WEIGHT|MOTHER_WEIGHT_GAIN|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
|                   0.0|          1|            29.0|               99.0|       0.0|      0.0|      0.0|      0.0|            99.0|            999.0|                 999.0|              99.0|         0.0| 

In [2]:
births = births.withColumn('BIRTH_PLACE_INT', births['BIRTH_PLACE']\
                           .cast(typ.IntegerType()))
births.printSchema()

root
 |-- INFANT_ALIVE_AT_REPORT: double (nullable = true)
 |-- BIRTH_PLACE: string (nullable = true)
 |-- MOTHER_AGE_YEARS: double (nullable = true)
 |-- FATHER_COMBINED_AGE: double (nullable = true)
 |-- CIG_BEFORE: double (nullable = true)
 |-- CIG_1_TRI: double (nullable = true)
 |-- CIG_2_TRI: double (nullable = true)
 |-- CIG_3_TRI: double (nullable = true)
 |-- MOTHER_HEIGHT_IN: double (nullable = true)
 |-- MOTHER_PRE_WEIGHT: double (nullable = true)
 |-- MOTHER_DELIVERY_WEIGHT: double (nullable = true)
 |-- MOTHER_WEIGHT_GAIN: double (nullable = true)
 |-- DIABETES_PRE: double (nullable = true)
 |-- DIABETES_GEST: double (nullable = true)
 |-- HYP_TENS_PRE: double (nullable = true)
 |-- HYP_TENS_GEST: double (nullable = true)
 |-- PREV_BIRTH_PRETERM: double (nullable = true)
 |-- BIRTH_PLACE_INT: integer (nullable = true)



In [22]:
import pyspark.ml.feature as ft

# Using the OneHotEncoder to encode
encoder = ft.OneHotEncoder(
inputCol='BIRTH_PLACE_INT',
outputCol='BIRTH_PLACE_VEC')
encoder.transform(births).show(2)
# featuresCreator = ft.VectorAssembler(
# inputCols=[col[0] for col in labels[2:]] + [encoder.getOutputCol()]\
#     ,outputCol='features')


# import pyspark.ml.classification as cl
# logistic = cl.LogisticRegression(maxIter=10,regParam=0.01\
#                                  ,labelCol='INFANT_ALIVE_AT_REPORT')

# from pyspark.ml import Pipeline
# pipeline = Pipeline(stages=[encoder,featuresCreator,logistic])
# births_train, births_test = births.randomSplit([0.7, 0.3], seed=666)
# pipelineModel = pipeline.fit(births_train)
# predictions = pipelineModel.transform(births_test)
# births_test.printSchema()

+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+---------------+---------------+
|INFANT_ALIVE_AT_REPORT|BIRTH_PLACE|MOTHER_AGE_YEARS|FATHER_COMBINED_AGE|CIG_BEFORE|CIG_1_TRI|CIG_2_TRI|CIG_3_TRI|MOTHER_HEIGHT_IN|MOTHER_PRE_WEIGHT|MOTHER_DELIVERY_WEIGHT|MOTHER_WEIGHT_GAIN|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|BIRTH_PLACE_INT|BIRTH_PLACE_VEC|
+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+---------------+---------------+
|                   0.0|          1|            29.0|               99.0|       0.0|      0.0|      0.0|     

In [4]:
# 评估模型的性能
import pyspark.ml.evaluation as ev

# 使用BinaryClassificationEvaluator来检验模型的表现，
# rawPredictionCol可以是由评估器产生的rawPrediction列，也可以是probability列
evaluator = ev.BinaryClassificationEvaluator(
rawPredictionCol='probability',
labelCol='INFANT_ALIVE_AT_REPORT')

print(evaluator.evaluate(predictions,{evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(predictions,{evaluator.metricName: 'areaUnderPR'}))

0.7342616552067652
0.587468578936341


In [15]:
test = spark.read.csv('inputData/births_test.csv',header=True,schema=schema)
test1.printSchema()

root
 |-- INFANT_ALIVE_AT_REPORT: double (nullable = true)
 |-- BIRTH_PLACE: string (nullable = true)
 |-- MOTHER_AGE_YEARS: double (nullable = true)
 |-- FATHER_COMBINED_AGE: double (nullable = true)
 |-- CIG_BEFORE: double (nullable = true)
 |-- CIG_1_TRI: double (nullable = true)
 |-- CIG_2_TRI: double (nullable = true)
 |-- CIG_3_TRI: double (nullable = true)
 |-- MOTHER_HEIGHT_IN: double (nullable = true)
 |-- MOTHER_PRE_WEIGHT: double (nullable = true)
 |-- MOTHER_DELIVERY_WEIGHT: double (nullable = true)
 |-- MOTHER_WEIGHT_GAIN: double (nullable = true)
 |-- DIABETES_PRE: double (nullable = true)
 |-- DIABETES_GEST: double (nullable = true)
 |-- HYP_TENS_PRE: double (nullable = true)
 |-- HYP_TENS_GEST: double (nullable = true)
 |-- PREV_BIRTH_PRETERM: double (nullable = true)
 |-- BIRTH_PLACE_INT: integer (nullable = true)



In [12]:
test1 = test.withColumn('BIRTH_PLACE_INT', test['BIRTH_PLACE']\
                           .cast(typ.IntegerType()))

10000

In [10]:
predictions1 = pipelineModel.transform(test1)
predictions1.show(1)

Py4JJavaError: An error occurred while calling o305.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 36, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (vector) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:234)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:827)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartiti